In [3]:
import pandas as pd
import os

# Paths
data_path = "../data"
output_path = "../cleaned"
os.makedirs(output_path, exist_ok=True)

# Mapping of xlsx files to csv filenames
files = {
    "coverage-data.xlsx": "coverage_data.csv",
    "incidence-rate-data.xlsx": "incidence_rate.csv",
    "reported-cases-data.xlsx": "reported_cases.csv",
    "vaccine-introduction-data.xlsx": "vaccine_intro.csv",
    "vaccine-schedule-data.xlsx": "vaccine_schedule.csv"
}

# Convert all XLSX files to CSV
for xlsx_file, csv_file in files.items():
    file_path = os.path.join(data_path, xlsx_file)
    df = pd.read_excel(file_path, engine="openpyxl")
    df.to_csv(os.path.join(output_path, csv_file), index=False)
    print(f"✅ Converted {xlsx_file} → {csv_file}")

print("\n All Excel files converted to CSV successfully!")


✅ Converted coverage-data.xlsx → coverage_data.csv
✅ Converted incidence-rate-data.xlsx → incidence_rate.csv
✅ Converted reported-cases-data.xlsx → reported_cases.csv
✅ Converted vaccine-introduction-data.xlsx → vaccine_intro.csv
✅ Converted vaccine-schedule-data.xlsx → vaccine_schedule.csv

 All Excel files converted to CSV successfully!


In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Visualization settings
sns.set(style="whitegrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)


In [34]:
# Use cleaned CSV files instead of Excel
data_path = "../cleaned"   # or "../data" if  haven't cleaned yet

df_coverage = pd.read_csv(os.path.join(data_path, "coverage_data.csv"))
df_incidence = pd.read_csv(os.path.join(data_path, "incidence_rate.csv"))
df_cases = pd.read_csv(os.path.join(data_path, "reported_cases.csv"))
df_intro = pd.read_csv(os.path.join(data_path, "vaccine_intro.csv"))
df_schedule = pd.read_csv(os.path.join(data_path, "vaccine_schedule.csv"))

print(" Datasets Loaded Successfully!")


 Datasets Loaded Successfully!


In [35]:
data_path = "../cleaned"   # or wherever your data is stored
df_coverage = pd.read_csv(os.path.join(data_path, "coverage_data.csv"))

# Basic overview
print("Shape:", df_coverage.shape)
df_coverage.head()


Shape: (399859, 11)


,GROUP,CODE,NAME,YEAR,ANTIGEN,ANTIGEN_DESCRIPTION,COVERAGE_CATEGORY,COVERAGE_CATEGORY_DESCRIPTION,TARGET_NUMBER,DOSES,COVERAGE
0,COUNTRIES,ABW,Aruba,2023.0,BCG,BCG,ADMIN,Administrative coverage,NaN,NaN,NaN
1,COUNTRIES,ABW,Aruba,2023.0,BCG,BCG,OFFICIAL,Official coverage,NaN,NaN,NaN
2,COUNTRIES,ABW,Aruba,2023.0,DIPHCV4,"Diphtheria-containing vaccine, 4th dose (1st b...",ADMIN,Administrative coverage,1044.0,945.0,90.52
3,COUNTRIES,ABW,Aruba,2023.0,DIPHCV4,"Diphtheria-containing vaccine, 4th dose (1st b...",OFFICIAL,Official coverage,NaN,NaN,90.52
4,COUNTRIES,ABW,Aruba,2023.0,DIPHCV5,"Diphtheria-containing vaccine, 5th dose (2nd b...",ADMIN,Administrative coverage,1219.0,1008.0,82.69


In [39]:
# Check which unique CODEs have missing NAMEs
codes_with_missing_names = df_coverage.loc[df_coverage['NAME'].isna(), 'CODE'].unique()
print("\nUnique CODEs with missing NAMEs:", codes_with_missing_names)

# We know 'WB_LONG_NA' and 'WB_SHORT_NA' are aggregate regions, not countries.
# Option → Assign a placeholder name instead of dropping
df_coverage.loc[df_coverage['CODE'] == "WB_LONG_NA", "NAME"] = "World Bank Long-term Aggregate"
df_coverage.loc[df_coverage['CODE'] == "WB_SHORT_NA", "NAME"] = "World Bank Short-term Aggregate"

# Drop rows where CODE is missing (only 1 row)
df_coverage = df_coverage.dropna(subset=['CODE'])

# -------------------------------
# 4. Handle Missing Values in Numeric Columns
# -------------------------------

# For TARGET_NUMBER and DOSES, fill missing values with median (more robust than mean)
df_coverage['TARGET_NUMBER'] = df_coverage['TARGET_NUMBER'].fillna(df_coverage['TARGET_NUMBER'].median())
df_coverage['DOSES'] = df_coverage['DOSES'].fillna(df_coverage['DOSES'].median())

# For COVERAGE (%), if missing → estimate it from DOSES / TARGET_NUMBER * 100
df_coverage['COVERAGE'] = df_coverage['COVERAGE'].fillna(
    (df_coverage['DOSES'] / df_coverage['TARGET_NUMBER']) * 100
)

# If still missing (e.g., TARGET_NUMBER = 0), fill with median COVERAGE
df_coverage['COVERAGE'] = df_coverage['COVERAGE'].fillna(df_coverage['COVERAGE'].median())

# -------------------------------
# 5. Handle Missing Values in Categorical Columns
# -------------------------------

# Fill missing ANTIGEN, ANTIGEN_DESCRIPTION, and COVERAGE_CATEGORY using mode
categorical_cols = ['ANTIGEN', 'ANTIGEN_DESCRIPTION', 'COVERAGE_CATEGORY', 'COVERAGE_CATEGORY_DESCRIPTION']
for col in categorical_cols:
    df_coverage[col] = df_coverage[col].fillna(df_coverage[col].mode()[0])

# -------------------------------
# 6. Normalize Data Types
# -------------------------------

# Ensure YEAR is integer
df_coverage['YEAR'] = df_coverage['YEAR'].astype(int)

# Ensure numeric columns are numeric
df_coverage['TARGET_NUMBER'] = pd.to_numeric(df_coverage['TARGET_NUMBER'], errors='coerce')
df_coverage['DOSES'] = pd.to_numeric(df_coverage['DOSES'], errors='coerce')
df_coverage['COVERAGE'] = pd.to_numeric(df_coverage['COVERAGE'], errors='coerce')

# -------------------------------
# 7. Remove Duplicates (if any)
# -------------------------------
df_coverage.drop_duplicates(inplace=True)

# -------------------------------
# 8. Final Check
# -------------------------------
print("\nShape after cleaning:", df_coverage.shape)
print("Missing values after cleaning:\n", df_coverage.isna().sum())

# -------------------------------
# 9. Save Cleaned Data
# -------------------------------
cleaned_path = "../cleaned"
df_coverage.to_csv(f"{cleaned_path}/coverage_data_cleaned.csv", index=False)
print("\n Coverage data cleaned and saved successfully!")


Unique CODEs with missing NAMEs: ['WB_LONG_NA' 'WB_SHORT_NA' nan]

Shape after cleaning: (399858, 11)
Missing values after cleaning:
 GROUP                            0
CODE                             0
NAME                             0
YEAR                             0
ANTIGEN                          0
ANTIGEN_DESCRIPTION              0
COVERAGE_CATEGORY                0
COVERAGE_CATEGORY_DESCRIPTION    0
TARGET_NUMBER                    0
DOSES                            0
COVERAGE                         0
dtype: int64

 Coverage data cleaned and saved successfully!


In [42]:
# Check how many rows have negative values
neg_rows = df_coverage[(df_coverage['DOSES'] < 0) | (df_coverage['TARGET_NUMBER'] < 0)]
print("🔍 Rows with negative values:\n", neg_rows)

# Drop rows with negative DOSES or TARGET_NUMBER
df_coverage = df_coverage[(df_coverage['DOSES'] >= 0) & (df_coverage['TARGET_NUMBER'] >= 0)]

print(f"\n Dropped {len(neg_rows)} rows with invalid negative values.")
print("New dataset shape:", df_coverage.shape)



🔍 Rows with negative values:
             GROUP CODE                 NAME  YEAR          ANTIGEN          ANTIGEN_DESCRIPTION COVERAGE_CATEGORY COVERAGE_CATEGORY_DESCRIPTION  TARGET_NUMBER        DOSES   COVERAGE
35587   COUNTRIES  BHS              Bahamas  2023  FLU_CHRONIC_PED  Influenza chronic pediatric             ADMIN       Administrative coverage       317871.5      -3333.0  -1.048537
35693   COUNTRIES  BHS              Bahamas  2022  FLU_CHRONIC_PED  Influenza chronic pediatric             ADMIN       Administrative coverage       317871.5      -3333.0  -1.048537
35800   COUNTRIES  BHS              Bahamas  2021  FLU_CHRONIC_PED  Influenza chronic pediatric             ADMIN       Administrative coverage       317871.5      -3333.0  -1.048537
280812  COUNTRIES  PRY             Paraguay  2022  FLU_CHRONIC_PED  Influenza chronic pediatric             ADMIN       Administrative coverage       317871.5      -3333.0  -1.048537
304668  COUNTRIES  SLV          El Salvador  2017      

In [43]:
# Find rows where coverage > 100%
over_coverage = df_coverage[df_coverage['COVERAGE'] > 100]
print(f"Rows where coverage > 100%: {len(over_coverage)}")
display(over_coverage.head())


Rows where coverage > 100%: 6097


,GROUP,CODE,NAME,YEAR,ANTIGEN,ANTIGEN_DESCRIPTION,COVERAGE_CATEGORY,COVERAGE_CATEGORY_DESCRIPTION,TARGET_NUMBER,DOSES,COVERAGE
1247,COUNTRIES,AFG,Afghanistan,2015,DTPCV1,"DTP-containing vaccine, 1st dose",ADMIN,Administrative coverage,1429729.0,1593629.0,111.00
1248,COUNTRIES,AFG,Afghanistan,2015,DTPCV1,"DTP-containing vaccine, 1st dose",OFFICIAL,Official coverage,317871.5,152212.0,109.00
1275,COUNTRIES,AFG,Afghanistan,2015,PCV1,"Pneumococcal conjugate vaccine, 1st dose",ADMIN,Administrative coverage,1429729.0,1618719.0,113.00
1276,COUNTRIES,AFG,Afghanistan,2015,PCV1,"Pneumococcal conjugate vaccine, 1st dose",OFFICIAL,Official coverage,317871.5,152212.0,111.00
1307,COUNTRIES,AFG,Afghanistan,2014,BCG,BCG,ADMIN,Administrative coverage,1512698.0,1530282.0,101.16


In [44]:
# Recalculate coverage
df_coverage['COVERAGE'] = (df_coverage['DOSES'] / df_coverage['TARGET_NUMBER']) * 100

# If target number is zero, set coverage to zero
df_coverage.loc[df_coverage['TARGET_NUMBER'] == 0, 'COVERAGE'] = 0


In [47]:
print("Rows with coverage > 100 after recalculation:", (df_coverage['COVERAGE'] > 100).sum())


Rows with coverage > 100 after recalculation: 0


In [ ]:
# Cap Coverage at 100% (Safe for dashboards)
df_coverage.loc[df_coverage['COVERAGE'] > 100, 'COVERAGE'] = 100
